### MEPS Workshop, April 14-15, 2020
### Analyzing MEPS-HC Data with SAS® 9.4M6 
#### By Pradip K. Muhuri


[Read about "Download MEPS-HC Transport Files, Documentation, and Codebooks"](https://meps.ahrq.gov/data_stats/download_data_files.jsp)

#### SAS Transport Files -  best overall format
* machine-independent (data files can be moved between computers running different operating systems). 

* can be directly imported into SAS, SPSS, BMDP, and STATA, etc.


#### Objectives
```
Run a SAS macro to automate the downloading and unzipping of the SAS transport files and converting of the transport files into SAS data sets.

Verify the following SAS data sets in your desired folder:

2017 MEPS Full-Year Consolidated File
2016 MEPS Full-Year Consolidated File
2017 Prescribed Medicine File
Panel 21 Longitudinal File 
Panel 20 Longitudinal File 
Panel 19 Longitudinal File 
```

This SAS program in the next code cell automates the:
* downloading of multiple SAS transport files from the MEPS website
* unzipping of those files
* converting of the unzipped transport files into SAS data sets

Before running the program, you need to change the specification for the:
  * FILENAME statement
  - LIBNAME statement
  
Written by: Pradip Muhuri  
Acknowledgements: Thanks to SAS(R) Institute for providing technical support 
to the revision/expansion of the original program.

##### Step 1:  Compile the following macro (SAS program)

In [17]:
options nonotes nodate nonumber; 
%macro load_MEPS(filename);
    filename inzip1 "C:\Data\&filename.ssp.zip";
    proc http 
     url="https://meps.ahrq.gov/data_files/pufs/&filename.ssp.zip"  
     out=inzip1;
    run;
/*From: https://blogs.sas.com/content/sasdummy/2015/05/11/using-filename-zip-to-unzip-and-read-data-files-in-sas/ 
*/
   filename inzip2 zip "c:\Data\&filename.ssp.zip" ;
   /* Read the "members" (files) from the ZIP file */
   Data contents(keep=memname isFolder);
       length memname $200 isFolder 8;
       fid=dopen("inzip2");
       if fid=0 then stop;
       memcount=dnum(fid);
       do i=1 to memcount;
             memname=dread(fid,i);
          /* check for trailing / in folder name */
          isFolder = (first(reverse(trim(memname)))='/');
        output;
       end;
     rc=dclose(fid);
     /* this could be automated if more than one file is expected in a zip */
     call symputx('memname',memname);
     run;
     %PUT &=MEMNAME;
    /* create a report of the ZIP contents */
    title "Files in the ZIP file";
     proc print data=contents noobs N;
     run;

    /* identify a temp folder in the WORK directory */
      filename sit "%sysfunc(getoption(work))/&memname." ;
 
    /* hat tip: "data _null_" on SAS-L */
    data _null_;
     /* using member syntax here */
      infile inzip2(&memname.) 
        lrecl=256 recfm=F length=length eof=eof unbuf;
    file sit lrecl=256 recfm=N;
    input;
    put _infile_ $varying256. length;
    return;
  eof:
    stop;
 run;
  libname test xport "%sysfunc(getoption(work))/&memname.";
  libname myfolder "C:\Data";  /* User-specified folder name */
 
     /*verify folder names*/
     %PUT %SYSFUNC(PATHNAME(WORK));
     %PUT %SYSFUNC(PATHNAME(MyFolder)); 
     proc copy in=test out=Myfolder; 
     run;
%mend load_MEPS;


#### Step 2: Invoke the macro with macro calls (SAS program) in the next cell.
```
There are six macro calls in the next code cell. From the MEPS website, they download/unzip selected SAS transport files and then convert them into SAS datasets in a pernament folder:

   a) 2017 full-year consolidated file
   c) 2016 full-year consolidated file
   d) Data for Panel 21 (2016-17)
   e) Data for Panel 19 (2015-16)
   f) Data for Panel 18 (2014-15)
   g) Precribed medicines file for 2017 (h197a)
 ```

In [18]:
%load_MEPS(h201)
%load_MEPS(h192)
%load_MEPS(h202)
%load_MEPS(h193)
%load_MEPS(h183)
%load_MEPS(h197a)

[Read this document: List all files within a directory including sub-directories](https://support.sas.com/kb/45/805.html)

The following SAS code snippet lists out the name of the SAS datasets
in the folder C:\Data.


In [19]:
 options nodate nonumber nonotes nosource;
 ods html close;
 Filename filelist pipe "dir /b /s c:\Data\*.SAS7bdat";
 Data _null_;                                        
     Infile filelist truncover;
     Input filename $100.;
     Put filename=;
   Run; 